# Recipe-LM: Fine-tune Gemma-2B with Unsloth

This notebook fine-tunes **Gemma-2B** on recipe data using [Unsloth](https://unsloth.ai/) for 2x faster training with 70% less VRAM.

**Requirements:** Google Colab with a T4 GPU runtime (free tier works).

1. Go to **Runtime > Change runtime type > T4 GPU**
2. Run all cells

## 1. Install Dependencies

In [1]:
!pip install unsloth
!pip install --no-deps trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7

## 2. Configuration

All training parameters in one place. These match the defaults from the recipe-lm pipeline.

In [2]:
# Model
MODEL_NAME = "unsloth/gemma-2b"  # Unsloth's optimized Gemma-2B
MAX_SEQ_LENGTH = 512
LOAD_IN_4BIT = True

# Dataset
DATASET_NAME = "corbt/all-recipes"
TEXT_COLUMN = "input"
MAX_SAMPLES = 5000
VAL_SPLIT_RATIO = 0.1

# LoRA
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

# Training
NUM_EPOCHS = 3
BATCH_SIZE = 4
LEARNING_RATE = 2e-4
SEED = 42

# Output
OUTPUT_DIR = "./lora_adapter"

## 3. Load Model with Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=LOAD_IN_4BIT,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id

print(f"Model loaded: {MODEL_NAME}")
print(f"Pad token: {tokenizer.pad_token}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## 4. Add LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
    max_seq_length=MAX_SEQ_LENGTH,
)

model.print_trainable_parameters()

## 5. Load & Prepare Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset(DATASET_NAME, split="train")
print(f"Full dataset: {len(dataset)} examples")

# Sample down
if MAX_SAMPLES and len(dataset) > MAX_SAMPLES:
    dataset = dataset.shuffle(seed=SEED).select(range(MAX_SAMPLES))
    print(f"Sampled to {len(dataset)} examples")

# Clean: strip whitespace, remove empty rows, deduplicate
def clean(example):
    text = example[TEXT_COLUMN]
    example[TEXT_COLUMN] = text.strip() if isinstance(text, str) else ""
    return example

dataset = dataset.map(clean)
dataset = dataset.filter(lambda x: len(x[TEXT_COLUMN]) > 0)

# Deduplicate
seen = set()
def dedup(example):
    text = example[TEXT_COLUMN]
    if text in seen:
        return False
    seen.add(text)
    return True

dataset = dataset.filter(dedup)
print(f"After cleaning: {len(dataset)} examples")

# Train/val split
split = dataset.train_test_split(test_size=VAL_SPLIT_RATIO, seed=SEED)
train_dataset = split["train"]
val_dataset = split["test"]
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

## 6. Train with SFTTrainer

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=SFTConfig(
        dataset_text_field=TEXT_COLUMN,
        max_seq_length=MAX_SEQ_LENGTH,
        packing=False,
        output_dir=OUTPUT_DIR,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        warmup_steps=5,
        seed=SEED,
        report_to="none",
    ),
)

print("Starting training...")
train_result = trainer.train()

## 7. Evaluate

In [ ]:
import math

train_loss = train_result.metrics["train_loss"]
eval_metrics = trainer.evaluate()
val_loss = eval_metrics["eval_loss"]
val_perplexity = math.exp(val_loss)

print(f"Train loss:      {train_loss:.4f}")
print(f"Val loss:        {val_loss:.4f}")
print(f"Val perplexity:  {val_perplexity:.2f}")

## 8. Save LoRA Adapter

In [ ]:
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Saved LoRA adapter to {OUTPUT_DIR}")

### (Optional) Push to HuggingFace Hub

Uncomment and fill in your details to upload the adapter.

In [ ]:
# from huggingface_hub import login
# login(token="hf_YOUR_TOKEN_HERE")
#
# HF_REPO = "your-username/gemma-2b-recipes-lora"
# model.push_to_hub(HF_REPO)
# tokenizer.push_to_hub(HF_REPO)
# print(f"Pushed to https://huggingface.co/{HF_REPO}")

## 9. Test Inference

In [ ]:
FastLanguageModel.for_inference(model)

prompts = [
    "Recipe for chocolate chip cookies:\n",
    "Recipe for pasta carbonara:\n",
    "Recipe for banana bread:\n",
]

for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"{'='*60}")
    print(text)
    print()

## 10. Download Adapter

Run this cell to download the adapter as a zip file, then use it locally:
```bash
python inference.py --adapter ./lora_adapter --prompt "Recipe for soup:"
```

In [ ]:
import shutil
from google.colab import files

shutil.make_archive("lora_adapter", "zip", OUTPUT_DIR)
files.download("lora_adapter.zip")